In [1]:
import onnx
import onnx_graphsurgeon as gs
import numpy as np
import onnxsim
from onnx import shape_inference,numpy_helper
from collections import OrderedDict

In [2]:
int64attr = OrderedDict([('to', 7)])
int32attr = OrderedDict([('to', 6)])
boolattr = OrderedDict([('to', 9)])
float32attr = OrderedDict([('to', 1)])
shape_dict = {"speech": [4, 64, 80],"speech_lengths": [4,]}

shape_infer = True
check = True

In [3]:
def check_gs(gs_graph):
    out_onnx = gs.export_onnx(gs_graph)
    out_onnx = shape_inference.infer_shapes(out_onnx)
    onnx.checker.check_model(out_onnx)
    return out_onnx

def findSlice(node):
    Slice = None
    try:
        for i in range(20):
            nextop = node.o(i)
    except Exception as e:
        return Slice
    else:
        return node

In [4]:
# !cd ../ && trtexec --workspace=8192 \
#  --onnx=./orin_onnx/encoder.onnx \
#  --optShapes=speech:4x64x80,speech_lengths:4 \
#  --maxShapes=speech:16x256x80,speech_lengths:16 \
#  --minShapes=speech:1x16x80,speech_lengths:1 \
#  --shapes=speech:1x16x80,speech_lengths:1 \
#  --saveEngine=./engine/encoder.plan \
#  2>&1 | tee ./log/encoder.log

In [4]:
ox = '../orin_onnx/encoder.onnx'

In [5]:
onnx_graph = onnx.load(ox)
gs_graph = gs.import_onnx(onnx_graph)

In [6]:
def deal_slice(gs_graph):
    Unsqueeze = Not = Slice = None
    for node in gs_graph.nodes:
        if node.op == 'Unsqueeze' and node.o().op == 'Not' and node.o().o().op == 'Slice':
            Unsqueeze = node
            Not = node.o()
            Slice = node.o().o()
    return Unsqueeze,Not,Slice

In [7]:
Unsqueeze,Not,Slice = deal_slice(gs_graph)

In [8]:
Cast0_input = Not.outputs[0]
Cast0_output =  gs.Variable(name="Cast0_output", dtype=np.dtype(np.int32), shape=None)

Cast0 = gs.Node(name='Add_Cast0',op='Cast',
               inputs=[Cast0_input],
               outputs=[Cast0_output],
               attrs=int32attr)
gs_graph.nodes.append(Cast0)

In [9]:
Slice.inputs[0] = Cast0_output

In [10]:
Slice0_output = Slice.outputs[0].copy()
Slice0_output.name = 'Slice0_output'

Cast1_input = Slice0_output
Cast1_output = Slice.outputs[0]

Slice.outputs[0] = Slice0_output

Cast1 = gs.Node(name='Add_Cast1',op='Cast',
               inputs=[Cast1_input],
               outputs=[Cast1_output],
               attrs=boolattr)
gs_graph.nodes.append(Cast1)

In [11]:
gs_graph.cleanup().toposort()
onnx.save(gs.export_onnx(gs_graph),'../tmp_onnx/encoder_fix_slice.onnx')

In [17]:
# !cd ../ && trtexec --workspace=8192 \
#  --onnx=./tmp_onnx/encoder.onnx \
#  --optShapes=speech:4x64x80,speech_lengths:4 \
#  --maxShapes=speech:16x256x80,speech_lengths:16 \
#  --minShapes=speech:1x16x80,speech_lengths:1 \
#  --shapes=speech:1x16x80,speech_lengths:1 \
#  --saveEngine=./engine/encoder.plan \
#  2>&1 | tee ./log/encoder.log

&&&& RUNNING TensorRT.trtexec [TensorRT v8401] # trtexec --workspace=8192 --onnx=./tmp_onnx/encoder.onnx --optShapes=speech:4x64x80,speech_lengths:4 --maxShapes=speech:16x256x80,speech_lengths:16 --minShapes=speech:1x16x80,speech_lengths:1 --shapes=speech:1x16x80,speech_lengths:1 --saveEngine=./engine/encoder.plan
[06/18/2022-09:54:45] [W] --workspace flag has been deprecated by --memPoolSize flag.
[06/18/2022-09:54:45] [I] === Model Options ===
[06/18/2022-09:54:45] [I] Format: ONNX
[06/18/2022-09:54:45] [I] Model: ./tmp_onnx/encoder.onnx
[06/18/2022-09:54:45] [I] Output:
[06/18/2022-09:54:45] [I] === Build Options ===
[06/18/2022-09:54:45] [I] Max batch: explicit batch
[06/18/2022-09:54:45] [I] Memory Pools: workspace: 8192 MiB, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[06/18/2022-09:54:45] [I] minTiming: 1
[06/18/2022-09:54:45] [I] avgTiming: 8
[06/18/2022-09:54:45] [I] Precision: FP32
[06/18/2022-09:54:45] [I] LayerPrecisions: 
[06/18/2022-09:54:45] [I] Calib

In [12]:
Slice_x = None
for node in gs_graph.nodes:
    if node.op == 'Slice' and findSlice(node):
        Slice_x = node

In [13]:
Slice_x

Slice_59 (Slice)
	Inputs: [
		Variable (onnx::Slice_554): (shape=None, dtype=None)
		Constant (onnx::Slice_3311): (shape=[1], dtype=<class 'numpy.int64'>)
		Variable (onnx::Slice_560): (shape=None, dtype=None)
		Constant (onnx::Slice_3312): (shape=[1], dtype=<class 'numpy.int64'>)
		Variable (onnx::Slice_563): (shape=None, dtype=None)
	]
	Outputs: [
		Variable (input.19): (shape=None, dtype=None)
	]

In [14]:
table5000x256 = np.load('5000x256')[0]
table5000x256.shape

(5000, 256)

In [15]:
table5000x256[1,:10]

array([0.84147096, 0.54030234, 0.8019618 , 0.59737533, 0.76172036,
       0.6479059 , 0.7214141 , 0.6925039 , 0.68156135, 0.731761  ],
      dtype=float32)

In [16]:
sliceTensor = Slice_x.inputs[2]
sliceTensor

Variable (onnx::Slice_560): (shape=None, dtype=None)

In [17]:
zero = gs.Constant(name='Constant-0',values=np.array([0]))
one = gs.Constant(name='Constant-1',values=np.array([1]))
three = gs.Constant(name='Constant-3',values=np.array([3]))

In [18]:
j=0
for i in range(1,24,2):
    trashNode = Slice_x.o(i).o().o()
    factor256x256 = Slice_x.o(i).inputs[1].values
    newTable = table5000x256@factor256x256
    newTable = newTable.transpose().reshape(1,4,64,5000)
    constantData = gs.Constant(f'Data-{j}',np.ascontiguousarray(newTable))
    sliceV = gs.Variable(f'sliceData-{j}',np.dtype(np.float32),[1,4,64,'slice'])
    sliceN = gs.Node('Slice',f'SliceN-{j}',
                    inputs = [constantData,zero,sliceTensor,three,one],
                    outputs=[sliceV])
    j+=1
    gs_graph.nodes.append(sliceN)
    Slice_x.o(i).o().o().o().inputs[1] = sliceV
    trashNode.outputs.clear()

In [19]:
gs_graph.cleanup().toposort()
onnx.save(gs.export_onnx(gs_graph),'../tmp_onnx/encoder_fix_5000x256.onnx')